#### Challenge

In [ ]:
from Crypto.Random.random import getrandbits
import random
from utils import listener

SEED = getrandbits(32)
FLAG = b'crypto{????????????????????}'
q = 127


class Challenge():
    def __init__(self):
        self.before_input = f"Welcome to the LWE sample generator! Retrieve a sample using the 'get_sample' option, or reset the distribution using the 'reset' option.\n"
        self.rand = random.Random(SEED)

    def challenge(self, your_input):
        if not "option" in your_input:
            return {"error": "You must send an option to this server"}

        elif your_input["option"] == "reset":
            self.rand.seed(SEED)
            return {"success": "The distribution has been reset"}

        elif your_input["option"] == "get_sample":
            a = []
            for i in range(len(FLAG)):
                a.append(self.rand.randint(0, q - 1))

            e = self.rand.randint(-1, 1)

            self.rand.seed(getrandbits(32))
            if self.rand.randint(0, 1):
                a[self.rand.randint(0, len(a) - 1)] = self.rand.randint(0, q - 1)

            b = 0
            for (i, j) in zip(a, FLAG):
                b += i * j
            b += e
            b %= q

            return {"a": a, "b": b}

        else:
            return {"error": "Invalid option"}


import builtins; builtins.Challenge = Challenge # hack to enable challenge to be run locally, see https://cryptohack.org/faq/#listener
"""
When you connect, the 'challenge' function will be called on your JSON
input.
"""
listener.start_server(port=13390)


In [6]:
from sage.all import *
from pwn import *
from Crypto.Util.number import *
import json 

host = 'socket.cryptohack.org' 
port = 13390
r = remote(host, port)

q = 127
r.recvuntil(b"using the 'reset' option.\n")

payload = {
    "option": "get_sample"
}
payload = json.dumps(payload).encode()
r.sendline(payload)

response = json.loads(r.recvline().strip().decode())
a = response["a"]
b = int(response["b"])

n = len(a)
m = 2 * n 

rows = []
lhs = [] 

for i in range(m):
    payload = {
        "option": "get_sample"
    }
    payload = json.dumps(payload).encode()
    r.sendline(payload)

    response = json.loads(r.recvline().strip().decode())
    a = response["a"]
    b = int(response["b"])

    rows.append(a)
    lhs.append(b)

A = Matrix(ZZ, rows)
b = vector(ZZ, lhs)

M = Matrix(ZZ, m + n + 1, m)
for i in range(m):
    M[i, i] = q 

for i in range(m):
    for j in range(n):
        M[m + j, i] = A[i, j] 
    
for i in range(m):
    M[m + n, i] = -b[i] 

M = M.LLL()
# print(M)

for i in range(m + n + 1):
    found = False
    for j in range(m):
        if M[i, j] != 0:
            found = True 
    if found:
        e = M[i]
        break 

f = []
for i in range(n):
    f.append(-1 * e[i])

f = vector(GF(q), f)

rows = []
for i in range(n):
    rows.append(A[i])

A_ = Matrix(GF(q), rows)

rows = []
for i in range(n):
    rows.append(b[i])

b_ = vector(GF(q), rows)

x = b_ - f 
s = A_**(-1) * x 

flag = ''
for i in range(n):
    flag += chr(int(s[i]))

print(flag)


[x] Opening connection to socket.cryptohack.org on port 13390
[x] Opening connection to socket.cryptohack.org on port 13390: Trying 134.122.111.232
[+] Opening connection to socket.cryptohack.org on port 13390: Done
crypto{f4ult_4ttack5_0n_lw3}
